# Exploration of Kernel Hearding for collecting super samples of a distribution

## The problem(s)
Deterministic methods for numerically calculating mulit-dimensional integrals $I = \int f(x)p(x)dx$ is scale poorly as the number of dimensions increase. This problem occurs frequently in calculating expectation values and marginal distributions.

Random iid sampling of $p(x)$ and evaluating the integral as $I \approx \frac{1}{N} \sum f(x_i)$ scales in a dimension agnostic way and converges to the true value as $I\approx \frac{1}{\sqrt(N)}$. 

A second problem is the need to reduce the amount of memory required estimate $p(x)$ for future use. An algorithm that can better estimate $p(x)$ would help solve both of these problems.

Improving this rate of convergence is a goal of kernel herding which seeks take advantage of the assumption that $p(x)$ is smooth and sample it in an optimal way to produce a set of 'super samples' that

## Kernel herding
A newer method, kernel herding, promises to converge on an estimate of $I$ somewhere between $\frac{1}{\sqrt(N)}$ and $\frac{1}{N}$. Kernel herding assumes that $p(x)$ is 'smooth', and then detrministically and recursively picking samples from $p(x)$. Every new sample 'knows' about the previous samples and stays away from them while still faithfully representing $p(x)$.

This algorithm finds each 'super sample' $x_{ss,i}$ by finding the point that maximizes:
$x_{ss,i} = argmax_x(\int k(x,x')p(x')dx' - \frac{1}{i}\sum k(x,x_{ss,i-1})$)
where $k(.,.)$ is a suitable kernel. For this demonstration I use $k(x,x')=e^{-norm(x-x')/\gamma^2}$ with $\gamma = 1$.

I call the first term the expectation value of the kernel expKer = $\int k(x,x')p(x')dx'$ and can be viewed as a weight drawing a point near regions of high probability density, while the second term sumKer = $\frac{1}{i}\sum k(x,x_{ss,i-1})$ acts to repel each point from all previous points.

## Implementation here
Create a low dimensional gaussian mixture model.

Sample points from $p(x)$ as an estimate.

Select an arbitrary start point, I pick the origin.

Estimate the first term expKer = $\int k(x,x')p(x')dx' \approx \sum_j e^{-norm(x-x_j)^2/\gamma^2}$

Calculate the second term sumKer = $\sum_{ss,i} e^{-norm(x-x_{ss,i})^2/\gamma^2}$

Use gradient descent to find argmax

Repeat for $N$ points


## Tradeoffs and comments
For creating a set of super samples that optimally reduce the amount of memory needed to store a distribution, herding is a good idea as long as we can afford the calculation of the samples.

Each super sample needs an integration and optimization so I'm trading an iid Monte Carlo sampling integral calculation for another. This means the calculation of expKer needs to be much less costly than the sampling $f(x)$. Monte Carlo sampling is a much better idea when sampling $f(x)$ is cheap.

Gradient descent, as implemented here, isn't always robust.

Each calculation of $x_{ss,i}$ is an infinite memory process and required knowledge of all supersamples found so far, meaning paralellization is more difficult (impossible?) than uncorrelated Monte Carlo sampling.

## Set up Python the environment

In [ ]:
!pip install numpy matplotlib pandas pypr scipy mpld3 scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd #to plot the GMM
from scipy.optimize import minimize #to do grad descent to fing argmax
import time #time each iteration
%matplotlib inline
import mpld3 #for plot tools
mpld3.enable_notebook()

# Create, sample, and plot GMM

In [ ]:
from sklearn.mixture import GaussianMixture



def sampleFromGauss(numDim, numGauss, muScale, covScale, numSamples):

  # Initialize variables
  mu = muScale * (np.random.rand(numGauss, numDim) - 0.5)
  sigma = np.zeros((numGauss, numDim, numDim))
  for i in range(numGauss):
    cov = covScale * (np.random.rand(numDim, numDim) - 0.5 + 0.7 * np.identity(numDim))
    sigma[i] = np.dot(cov, cov.T)

  # Create Gaussian Mixture Model
  gmm = GaussianMixture(n_components=numGauss, covariance_type='full')
  gmm.means_ = mu
  gmm.covariances_ = sigma
  gmm.weights_ = np.ones(numGauss) / numGauss
  gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(sigma))

# Sample from GMM
  samples, _ = gmm.sample(numSamples)
  return samples, (gmm.weights_, mu, sigma), gmm

# Calculate herding samples

In [ ]:

def expKer(x, samples, gamma):
    """
    Calculates the expectation value of the exponential kernel to find argmax_x.
    Parameters:
    x (numpy.ndarray): Candidate super sample to optimize.
    samples (numpy.ndarray): The Gaussian Mixture Model (GMM) samples.
    gamma (float): Kernel hyperparameter, always 1 for the demo.
    Returns:
    float: The expectation value of the exponential kernel.
    """


    numSamples = samples.shape[0]
    k = np.exp(-np.linalg.norm(x - samples, axis=1) / gamma**2)
    exp_est = np.mean(k)
    return exp_est

def sumKer(x,xss,numSSsoFar,gamma):
    
    """
    Calculates the sum of the kernel function k(x, x_ss) for the given number of super samples so far.
    Parameters:
    x (numpy.ndarray): The candidate super sample to optimize.
    xss (numpy.ndarray): The array of super samples.
    numSSsoFar (int): The number of super samples so far.
    gamma (float): The kernel hyperparameter.
    Returns:
    float: The normalized sum of the kernel values.
    """    
    # Vectorized calculation of the sum of kernels
    k = np.exp(-np.linalg.norm(x - xss[:numSSsoFar], axis=1) / gamma**2)
    total = np.sum(k)
    s = total / (numSSsoFar + 1)
    return s





Sampling

In [ ]:
#create samples
np.random.seed([1])
samples, coeffs = sampleFromGauss(numDim=3, numGauss=4, muScale=5, covScale=1, numSamples=int(1e3))
#plot them for inspection
df = pd.DataFrame(samples)
pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(6, 6))

In [ ]:
#-- main function

def herd(samples,totalSS,gamma):
    #-- calculate totalSS super samples from the distribution estimated by samples with kernel hyperparam gamma
    
    #init vars and extract useful info from inputs
    #get GMM dims and num samples
    numDim = samples.shape[1]
    numSamples = samples.shape[0]
    
    #init vars
    gradientFail = 0; #count when optimization fails, debugging
    xss = np.zeros((totalSS,numDim)) #open space in mem for array of super samples
    i=1
    #gradient descent can have some probems, so make bounds to terminate if goes too far away
    minBound = np.min(samples)
    maxBound = np.max(samples)
    #start our search at the origin, could be a random point
    bestSeed = np.zeros(numDim)
    
#     tick = time.clock()
    while i<totalSS:
        # print('.'),
        #debugging stuff
        #print "Working on SS num ber i=%d" % i
        #build function for gradient descent to find best point
        f = lambda x: -expKer(x,samples,gamma)+sumKer(x,xss,i,gamma)
        results = minimize(f,
                   bestSeed,
                   method='L-BFGS-B',
                   bounds=[(minBound, maxBound)] * numDim,
                   options={'ftol': 1e-9, 'disp': False})
#         print "results.x"
#         print results.x
        
        #if grad descent failed, pick a random sample and try again
        if np.min(results.x) < minBound or np.max(results.x) > maxBound:
            bestSeed=samples[np.random.choice(numSamples)]
            gradientFail=gradientFail+1
#             print "Gradient descent failed.............."
            continue
        
        #pick next best start point to start minimization, this is how Chen, Welling, Smola do it
        #find best super sample that maximizes argmax and use that as a seed for the next search
        #init or clear seed array
        seed=np.array([])
        for j in range(i):
            seed = np.append(seed,-expKer(xss[j,:],samples,gamma)+sumKer(xss[j,:],xss,i,gamma)) # put f function values of xss's in seed array
        bestSeedIdx = np.argmin(seed)
        bestSeed=xss[bestSeedIdx,:]
        
        #grad descent succeeded (yay!), so assign new value to super samples
        xss[i,:]=results.x
        
        i=i+1
        # toc = time.clock()
#         print "Time elapsed %d" % (toc-tick)
    return xss

In [ ]:
from matplotlib.colors import LogNorm
from scipy.stats import multivariate_normal

def plot_gmm_density(coeffs, ax=None):
    weights, means, covariances = coeffs
    if ax is None:
        ax = plt.gca()
    
    x = np.linspace(-10, 10, 200)  # Increase the number of points
    y = np.linspace(-10, 10, 200)  # Increase the number of points
    X, Y = np.meshgrid(x, y)
    XX = np.array([X.ravel(), Y.ravel()]).T

    Z = np.zeros((200, 200))  # Adjust the shape accordingly
    for weight, mean, cov in zip(weights, means, covariances):
        rv = multivariate_normal(mean[:2], cov[:2, :2])
        Z += weight * rv.pdf(XX).reshape(200, 200)
    
    ax.contourf(X, Y, Z, levels=100, cmap='Blues', norm=LogNorm())
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.title('Gaussian Mixture Model Density')
    plt.colorbar(ax.contourf(X, Y, Z, levels=50, cmap='viridis', norm=LogNorm()))
    plt.show()


In [ ]:
totalSS=100
xss = herd(samples,totalSS,gamma=1)

In [ ]:
fig, ax = plt.subplots()

# Plot herd samples


# Plot random samples
ax.plot(samples[:, 0], samples[:, 1], '.', label='Random Samples')
ax.plot(xss[:, 0], xss[:, 1], 'o', label='Herd Samples')
# Plot the GT GMM showing the density with colors
plot_gmm_density(coeffs, ax=ax)

# Add legend
ax.legend()

plt.show()

# Plot first 2 dims of distribution and super samples

## Estimate error

Here I follow Chen, Welling, and Smola, and report the error as defined by equation 5:$err_N = norm(\mu_p - \frac{1}{N}\sum_i x_{ss,i})$. Essentially this compares the mean of the GMM to the mean estimated by super sampling. I compare this to the iid sampled mean.

In [ ]:
# Calculate herding error
mu_p = np.mean(samples,axis=0)
err=np.zeros(totalSS)
for i in range(totalSS):
    err[i]    = np.linalg.norm(mu_p-np.sum(xss[1:i,:]        ,axis=0)/i)

idx = np.random.choice(1000,totalSS)
samples_iid=samples[idx,:]
err_iid=np.zeros(totalSS)
for i in range(totalSS):
    err_iid[i]= np.linalg.norm(mu_p-np.sum(samples_iid[1:i,:],axis=0)/i)

In [ ]:
plt.plot(err,'-o')
plt.plot(err_iid)
plt.plot([1,totalSS],[1,1./totalSS],'--')
plt.yscale('log')
plt.xscale('log')
plt.legend(['Herding error','iid error','~1/N'], loc='lower left')